<a href="https://colab.research.google.com/github/F-Nourin369/Traffic-sign-classification-using-CNN/blob/main/inceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
pip install opencv-python


In [ ]:
# Importing Libraries
from tensorflow import keras
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

import os
import cv2
import PIL.Image
import numpy as np
import tensorflow as tf
import pickle
import matplotlib.pyplot as plt

set path to dataset

In [ ]:
train_dir='/content/drive/MyDrive/newtrafficsign/train'
validation_dir='/content/drive/MyDrive/newtrafficsign/test'

setting the parameters

In [ ]:
num_classes=85
image_size=(299,299)
batch_size=32
learning_rate=0.001
epochs=30

preprocessing and augment the training data

In [ ]:
train_datagen=ImageDataGenerator(
                 horizontal_flip=True,

                 rescale=1. / 255,

                 shear_range=0.2,
                 zoom_range=0.2,

)

preprocess the validation data

In [ ]:
valid_datagen=ImageDataGenerator(rescale=1 / 255,

                                 )

load the InceptionV3 model

In [ ]:
# Initializing InceptionV3 (pretrained) model with input image shape
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(image_size[0],image_size[1],3))


87910968/87910968 [==============================] - 1s 0us/step


adding the additional top layer for the model

In [ ]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x)
predictions=Dense(num_classes,activation='softmax')(x)

combine base model with  top layer

In [ ]:
model=Model(inputs=base_model.input,outputs=predictions)

Freeze the layers in base model

In [ ]:
for layer in base_model.layers:
  layer.trainable=False

compile the model

In [ ]:
model.compile(optimizer=Adam(learning_rate=learning_rate),loss='categorical_crossentropy',metrics=['accuracy'])

generate the training and validation data from directories

In [ ]:

train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

valid_generator=valid_datagen.flow_from_directory(
    validation_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 4438 images belonging to 85 classes.
Found 1288 images belonging to 85 classes.


train the model

In [ ]:
history=model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=valid_generator.n// batch_size
)

Epoch 1/30
138/138 [==============================] - 445s 3s/step - loss: 2.1962 - accuracy: 0.4444 - val_loss: 1.5528 - val_accuracy: 0.6195
Epoch 2/30
138/138 [==============================] - 180s 1s/step - loss: 1.1023 - accuracy: 0.6729 - val_loss: 1.1947 - val_accuracy: 0.6898
Epoch 3/30
138/138 [==============================] - 184s 1s/step - loss: 0.7849 - accuracy: 0.7671 - val_loss: 1.0258 - val_accuracy: 0.7133
Epoch 4/30
138/138 [==============================] - 174s 1s/step - loss: 0.6746 - accuracy: 0.7914 - val_loss: 0.9759 - val_accuracy: 0.7492
Epoch 5/30
138/138 [==============================] - 185s 1s/step - loss: 0.5579 - accuracy: 0.8173 - val_loss: 0.9786 - val_accuracy: 0.7227
Epoch 6/30
138/138 [==============================] - 179s 1s/step - loss: 0.4743 - accuracy: 0.8429 - val_loss: 1.0429 - val_accuracy: 0.7375
Epoch 7/30
138/138 [==============================] - 184s 1s/step - loss: 0.4429 - accuracy: 0.8493 - val_loss: 0.9091 - val_accuracy: 0.7625

save the trained model

In [ ]:
model.save('/content/drive/MyDrive/inceptionv3_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Testing the Model

In [3]:
!pip install gtts


In [ ]:
!pip install ipywebrtc

In [ ]:
pip install ipywidgets


In [ ]:
from ipywidgets import widgets


In [ ]:
!pip install ipywebrtc


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 5.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import PIL
import matplotlib.pyplot as plt
import tensorflow as tf
from google.colab import files
from gtts import gTTS
import ipywidgets as widgets
import IPython.display as ipd

# Load the model
model = tf.keras.models.load_model('/content/drive/MyDrive/inceptionv3_model.h5')

# Define class labels
class_labels = {0: 'ALL MOTOR VEHICLE PROHIBITED', 1: 'AXLE LOAD LIMIT', 2: 'BARRIER AHEAD', 3: 'BULLOCK AND HANDCART PROHIBITED', 4: 'Dragonfly', 5: 'CATTLE',
                6: 'COMPULSARY AHEAD', 7: 'COMPULSARY AHEAD OR TURN LEFT', 8: 'COMPULSARY AHEAD OR TURN RIGHT', 9: 'COMPULSARY CYCLE TRACK',10: 'COMPULSARY KEEP RIGHT',11: 'COMPULSARY KEEP RIGHT',12: 'COMPULSARY MINIMUM SPEED',13: 'COMPULSARY SOUND HORN',14: 'COMPULSARY TURN LEFT',
                15: 'COMPULSARY TURN LEFT AHEAD',16: 'COMPULSARY TURN RIGHT',17: 'COMPULSARY TURN RIGHT AHEAD',18: 'CROSS ROAD',19: 'CYCLE CROSSING',20: 'CYCLE PROHIBITED',21: 'DANGEROUS DIP',
                22: 'DIRECTION',23: 'FALLING ROCKS',24: 'FERRY',25: 'GAP IN MEDIAN',26: 'GIVE WAY',27: 'GUARDED LEVEL CROSSING',28: 'HANDCART PROHIBITED',
                29: 'HEIGHT LIMIT',30: 'HORN PROHIBITED',31: 'HUMP OR ROUGH ROAD',32: 'LEFT HAIR PIN BEND',33: 'LEFT HAND CURVE',34: 'LEFT REVERSE BEND',35: 'LEFT TURN PROHIBITED',
                36: 'LENGTH LIMIT',37: 'LOAD LIMIT',38: 'LOOSE GRAVEL',39: 'MEN AT WORK',40: 'NARROW BRIDGE',41: 'NARROW ROAD AHEAD',42: 'NO ENTRY',
                43: 'NO PARKING',44: 'NO STOPPING OR STANDING',45: 'OVERTAKING PROHIBITED',46: 'PASS EITHER SIDE',47: 'PEDESTRIAN CROSSING',48: 'PEDESTRIAN PROHIBITED',49: 'PRIORITY FOR ONCOMING VEHICLES',
                50: 'QUAY SIDE OR RIVER BANK',51: 'RESTRICTION ENDS',52: 'RIGHT HAIR PIN BEND',53: 'RIGHT HAND CURVE',54: 'RIGHT REVERSE BEND',55: 'RIGHT TURN PROHIBITED',56: 'ROAD WIDENS AHEAD',
                57: 'ROUNDABOUT',58: 'SCHOOL AHEAD',59: 'SIDE ROAD LEFT',60: 'SIDE ROAD RIGHT',61: 'SLIPPERY ROAD',62: 'SPEED LIMIT 15',63: 'SPEED LIMIT 20',
                64: 'SPEED LIMIT 30',65: 'SPEED LIMIT 40',66: 'SPEED LIMIT 5',67: 'SPEED LIMIT 50',68: 'SPEED LIMIT 60',69: 'SPEED LIMIT 70',70: 'SPEED LIMIT 80',
                71: 'STAGGERED INTERSECTION',72: 'STEEP ASCENT',73: 'STEEP DESCENT',74: 'STOP',75: 'STRAIGHT PROHIBITED',76: 'TONGA PROHIBITED',77: 'TRAFFIC SIGNAL',
                78: 'TRUCK PROHIBITED',79: 'TURN RIGHT',80: 'T INTERSECTION',81: 'UNGUARDED LEVEL CROSSING',82: 'U TURN PROHIBITED',83: 'WIDTH LIMIT',84: 'Y INTERSECTION'}

# Function to preprocess the image
def preprocess_image(image_path):
    img = PIL.Image.open(image_path)
    img = img.resize((299, 299))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# Function to predict the sign
def predict_sign(image_path):
    processed_image = preprocess_image(image_path)
    predictions = model.predict(processed_image)
    predicted_label = class_labels[np.argmax(predictions)]
    confidence = np.max(predictions)
    return predicted_label, confidence

# Function to handle file upload option
def upload_option(b):
    uploaded = files.upload()
    uploaded_file_path = list(uploaded.keys())[0]
    prediction, confidence = predict_sign(uploaded_file_path)
    img = PIL.Image.open(uploaded_file_path)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f'Prediction: {prediction}\nConfidence: {confidence:.2%}')
    plt.show()

    # Generate voice alert
    tts = gTTS(text=prediction, lang='en')
    tts.save("prediction.mp3")

    # Display voice alert
    ipd.display(ipd.Audio("prediction.mp3", autoplay=True))

# Create upload button
upload_button = widgets.Button(description="Upload")

# Define button click event
upload_button.on_click(upload_option)

# Display button
display(upload_button)


In [ ]:
!pip install opencv-python-headless
!pip install ipywebrtc
from google.colab import output
output.enable_custom_widget_manager()

import cv2
from google.colab.patches import cv2_imshow
from IPython.display import display, HTML
from ipywebrtc import CameraStream
import ipywidgets as widgets

# Define the function to handle file uploads
def handle_upload(change):
    img = change['new'][0]
    img_widget.value = img['content']

# Create a camera stream object
camera = CameraStream(constraints={'audio': False, 'video': True})

# Create a file upload widget
uploader = widgets.FileUpload(accept='.png,.jpg,.jpeg', multiple=False)

# Create an output widget to display images or videos
img_widget = display(None, display_id='image')

# Function to display the uploaded image
def display_image(change):
    img = change['new']
    img_widget.update(img)

# Link the file uploader to the display image function
uploader.observe(display_image, names='value')

# Display the camera stream and file uploader
display(HTML("<h1>Select an option:</h1>"))
display(HTML("<h2>Camera</h2>"))
display(camera)
display(HTML("<h2>Upload</h2>"))
display(uploader)
